In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
current_dir = os.getcwd()
dir_warehouse = f"{current_dir}/warehouse"

In [3]:
spark = SparkSession.builder \
    .appName("IcebergWithSpark") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", dir_warehouse) \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .getOrCreate()

25/01/07 21:25:16 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/01/07 21:25:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/07 21:25:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# !unzip ../Iceberg/vendas_iceberg.zip -d ./
# !mkdir -p ./warehouse/default/vendas_iceberg
# !cp -r ../Iceberg/vendas_iceberg/* ./warehouse/default/vendas_iceberg/

In [4]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/apolo/Dropbox/programacao/Udemy/2024/engenharia_de_dados_com_apache_iceberg_e_spark/00-scripts_apolo/spark-warehouse')]

In [5]:
spark.catalog.listTables(dbName="default")

[]

In [6]:
# Criar uma tabela Iceberg diretamente a partir do Parquet existente
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas_iceberg")

DataFrame[]

In [7]:
spark.sql("""
CREATE TABLE hadoop_catalog.default.vendas_iceberg (
    id INT,
    produto STRING,
    quantidade INT,
    preco DOUBLE,
    data_venda DATE
)
USING iceberg
""")

DataFrame[]

In [8]:
# le parquet
df_parquet = spark.read.parquet("./warehouse/default/vendas_iceberg/")

In [9]:
df_parquet.show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
+---+---------+----------+-----+----------+



In [10]:
# atualiza na tabela
df_parquet.writeTo("hadoop_catalog.default.vendas_iceberg").append()

In [11]:
# visualizamos os dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_iceberg order by id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2023-11-01|
|  2|Produto B|         5| 22.0|2023-11-02|
|  3|Produto C|         8| 30.0|2023-11-03|
|  4|Produto D|        12| 25.0|2023-11-04|
|  5|Produto E|         7| 18.5|2023-11-05|
+---+---------+----------+-----+----------+



In [12]:
# mostra snapshots
snapshots_df = spark.sql("SELECT * FROM hadoop_catalog.default.vendas_iceberg.snapshots")
snapshots_df.select("snapshot_id", "committed_at", "operation").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|9160510581618144367|2025-01-07 21:33:35.782|append   |
+-------------------+-----------------------+---------+

